# 제4장 Matrix Factorization(MF) 기반 추천

**메모리 기반 알고리즘** : 추천을 위한 데이터를 모두 메모리에 가지고 있으면서 추천이 필요할 때마다 이데이터를 사용해서 계산을 해서 추천하는 방식  
   - ex) CF

<br>

**모델 기반 추천** : 추천을 위한 모델을 구성한 후에 이 모델만 저장하고, 실제 추천을 할 때에는 이 모델을 사용해서 추천을 하는 방식  
   - ex) MF, Deep-Learning 방식의 추천도 데이터

<br>

메모리 기반 추천은 모든 데이터를 메모리에 저장하고 있기 때문에 원래 데이터를 충실하게 사용하는 장점이 있지만 대량의 데이터를 다뤄야 하는 상용 사이트에서는 계산시간이 너무 오래 걸린다는 단점이 있다  
이에 비해 모델 기반 추천 방식은 원래 데이터는 모형을 만드는 데만 사용하고 일단 모델이 만들어진면 원래 데이터는 사용하지 않기 때문에 대규모 상용 사이트에서 필요한 빠른 반응이 가능하지만 모델을 만드는 과정에서 많은 계산이 필요하다는 단점이 있다.  
일반적으로 메모리 기반 추천은 개별 사용자의 데이터에 집중하는 데 비해, 모델 기반 추천은 전체 사용자의 평가 패턴으로부터 모델을 구성하기 때문에 데이터가 가지고 있는 약한 신호 (weak signal)도 더 잘 잡아내는 장점이 있다.  
  - 약한신호 : 개별 사용자의 행동분석에서는 잘 드러나지 않는 패턴


## 4.1 Matrix Factorization(MF) 방식의 원리

<br>

행렬요인화는 평가 데이터, 즉 (사용자 x 아이템)으로 구성된 하나의 행렬을 2개의 행렬로 분해하는 방법


$R \approx P \times Q^T = \hat{R} $
- R : Rating matrix
- P : User latent matrix(사용자 잠재요인행렬)
- Q : Item latent matrix(아이템 잠재요인행렬)

<br>

MF 방식은 이 R행렬을 사용자행렬(P)과 아이템행렬(Q)로 쪼개어 분석하는 것  
- P는 (M x K), Q는 (N x K)  
- 여기서 $\hat{R}$은 R의 예측치이며 $\hat{R}$이 최대한 R에 가까운 값을 가지도록 하는 P와 Q를 구하면 그것이 바로 추천을 위한 모델이 된다
- P는 각 사용자의 특성을 나타내는 K개 요인의 값으로 이루어진 행렬, Q는 각 아이템의 특성을 나타내는 K개의 요인의 값으로 이루어진 행렬
- P와 Q행렬에서 공통인 K개의 요인이 있는데, 이를 잠재요인(latent factor)이라고 부른다.
- 즉 사용자와 아이템의 특성을 K개의 잠재요인을 사용해서 분석하는 모델이라고 한다



## 4.2 SGD(Stochastic Gradient Descent)를 사용한 MF 알고리즘

<br>

주어진 (사용자x아이템)의 평점행렬인 R로부터 P와Q를 분해하는 알고리즘
1. 잠재요인의 개수인 K를 정한다. K는 경험에 의해 직관적으로 정해도 되고 다양한 K를 비교하면서 최적의 수를 정해도 된다.
2. 주어진 K에 따라 P(MxK)와 Q(NxK)행렬을 만들고 초기화한다. 맨 처음에는 P,Q 행렬을 임의의 수로 채우는 것이 보통이다.
3. 주어진 P,Q 행렬을 사용해서 예측 평점 $\hat{R}(=P \times Q^T)$을 구한다
4. R에 있는 실제 평점에 대해서 예측 평점 $\hat{R}$의 예측과 비교해서 오차를 구하고, 이 오차를 줄이기 위해서 P,Q값을 수정한다
5. 전체 오차가 미리 정해진 기준값 이하가 되거나 미리 정해진 반복 횟수에 도달할 때까지 3번으로 돌아것 반복한다
<br>

여기서 핵심은 4번에서 예측 오차를 줄이기 위해서 P,Q를 어떻게 수정하는가이다  
가장 일반적인 방법은 기계학습에서 많이 사용되는 SGD(Stochastic Gradient Descent) 방법을 적용하는 것이다 

<br>


$L = \frac{1}{2} \sum_{(i,j) \in \Omega} \left( R_{ij} - P_i Q_j^\top \right)^2 + \frac{\lambda}{2} \left( \|P\|_F^2 + \|Q\|_F^2 \right)$

![확률적 경사하강법](image/이미지_4-1.png)

<br>

![정규화항](image/이미지_4-2.png)

- b : 전체 평균
- $bu_i$ : 전체 평균을 제거한 후 사용자 i의 평가경향(사용자 i의 평균과 전체 평균의 차이)
- $bd_j$ : 전체 평균을 제거한 후 아이템 j의 평가경향(아이템 j의 평균과 천체 평균의 차이)

## 4.3 SGD를 사용한 MF 기본 알고리즘


In [19]:
import pandas as pd
import numpy as np
from numpy.ma.core import nonzero
from sqlalchemy.dialects.mssql.information_schema import columns
from zmq import value

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('data/u.data',sep='\t',names=r_cols,encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

# MF class
class MF():
    def __init__(self,ratings,k,alpha,beta,iterations,verbose=True):
        self.R = np.array(ratings)
        self.num_users,self.num_items = np.shape(self.R)
        self.k = k
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose =verbose # SGD의 중간 학습과정을 출력할 것인가
        
    # Root Mean Squared Error(RMSE) 계산
    def rmse(self):
        # R에서 평점이 있는(0이 아닌) 요소의 인덱스를 가져온다
        xs,ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x,y in zip(xs,ys):
            prediction = self.get_prediction(x,y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x,y]-prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))
    
    # 정해진 반복 횟수만큼 앞의 식 2번,4번을 사용해서 P,Q,bu,bd 값을 업데이트하는 함수
    def train(self):
        # Initializing user-feature and movie-feature matrix
        # 행렬을 임의의 값으로 채운다. 여기서는 평균 0, 표준편차 1/K인 정규분포를 갖는 난수로 초기화한다.
        self.P = np.random.normal(scale=1./self.k,size=(self.num_users,self.k)) # K가 커질수록 행렬의 각 원소는 더 많은 잠재요인들과 조합, 이때 초기값의 범위를 너무 넓게 설정하면 모델의 표현 범위가 불필요하게 커져 학습 과정에서 불안정성이 증가 또 너무 작은 초기값을 가지면 학습 속도가 느려지고 모델이 충분히 다양한 방향으로 학습되지 않을 수 있다
        self.Q = np.random.normal(scale=1./self.k,size=(self.num_items,self.k))
        
        #Initializing the bias terms
        # bu,bd를 0으로 초기화
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        # 0이 아닌 전체 평점 평균을 b에 저장
        self.b = np.mean(self.R[self.R.nonzero()])
        
        # List of training samples
        rows,columns = self.R.nonzero()
        # 평점이 있는 요소의 인덱스, 평점을 리스트로 저장
        self.samples = [(i,j,self.R[i,j]) for i,j in zip(rows,columns)]
        
        # SGD for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse=self.rmse()
            training_process.append((i+1,rmse))
            if self.verbose:
                if (i+1)%10==0:
                    print("Iteration: %d ; Train RMSE = %.4f"%(i+1,rmse))
        return training_process

    # Rating prediction for user i and item j
    def get_prediction(self,i,j):
        prediction = self.b+self.b_u[i]+self.b_d[j]+self.P[i,:].dot(self.Q[j,:].T)
        return prediction
    
    # SGD to get optimized P and Q matrix
    def sgd(self):
        for i,j,r in self.samples:
            prediction = self.get_prediction(i,j)
            e = (r-prediction)
            
            # 앞의 4번 식을 적용해서 bu,bd 업데이트
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])
            
            # 앞의 2번 식을 적용해서 P,Q 업데이트
            self.P[i,:] += self.alpha*(e*self.Q[j,:] - self.beta*self.P[i,:])
            self.Q[j,:] += self.alpha*(e*self.P[i,:] - self.beta*self.Q[j,:])
    

In [20]:
# 전체 데이터 사용 MF
# dataframe 형식을 full matrix로 변환 -> MF 클래스 내부적으로 full matrix(self.R)를 계산에 사용하기 때문
R_temp = ratings.pivot(index = 'user_id',columns = 'movie_id',values = 'rating').fillna(0)
mf = MF(R_temp,k=30,alpha=0.001,beta=0.02,iterations=100,verbose=True)
train_process = mf.train()

Iteration: 10 ; Train RMSE = 0.9585
Iteration: 20 ; Train RMSE = 0.9373
Iteration: 30 ; Train RMSE = 0.9280
Iteration: 40 ; Train RMSE = 0.9225
Iteration: 50 ; Train RMSE = 0.9183
Iteration: 60 ; Train RMSE = 0.9144
Iteration: 70 ; Train RMSE = 0.9099
Iteration: 80 ; Train RMSE = 0.9037
Iteration: 90 ; Train RMSE = 0.8948
Iteration: 100 ; Train RMSE = 0.8828


## 4.4 train/test 분리 MF 알고리즘

<br>

CF에서는 sklearn의 train_test_split을 사용했는데 여기서는 sklearn의 shuffle을 사용

In [21]:
# train test 분리
from sklearn.utils import shuffle
TRAIN_SIZE = 0.75
# dataframe 형식으로 되어있는 ratings를 무작위로 섞는다
ratings = shuffle(ratings,random_state=1) # random_state : 랜덤 시드

# 전체 데이터 중 train 데이터 개수
cutoff = int(TRAIN_SIZE*len(ratings))
ratins_train = ratings.iloc[:cutoff] # iloc[] : 정수 기반 인덱싱, 행과 열의 숫자 위치로 기반
ratings_test = ratings.iloc[cutoff:]

user_id와 item_id를 각각의 인덱스와 매핑하는 user_id_index,item_id_index가 클래스 속성으로 추가된 것.  
이것이 필요한 이유는 user_id와 item_id가 내부의 인덱스와 일치하지 않기 때문이다.  
user_id와 item_id가 연속값이 아닐 수도 있다 이 경우 데이터는 클래스 내부에서 numpy array인 self.R로 변환되면서 중간이 비어있는 실제 아이디와 R의 인덱스가 일치하지 않게 된다. 왜냐하면 numpy array는 무조건 연속되는 값이 지정되지만 아이디는 그렇지 않기 때문
- 연속된 값 ex) 0,1,2 ...

In [25]:
# New MF class for training & testing
class NEW_MF():
    def __init__(self, ratings, k, alpha, beta, iterations, verbose=True):
        self.R = np.array(ratings)
        # user_id, item_id를 R의 index와 매핑하기 위한 dictionary
        item_id_index = []
        index_item_id = []
        for i,one_id in enumerate(ratings): # enumerate : (인덱스,값) 형식의 튜플을 생성하여 반환
            item_id_index.append([one_id,i])
            index_item_id.append([i,one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)
        
        user_id_index = []
        index_user_id = []
        for i,one_id in enumerate(ratings.T):
            user_id_index.append([one_id,i])
            index_user_id.append([i,one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)

        self.num_users, self.num_items = np.shape(self.R)
        self.k = k
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose
        
    # train set의 RMSE 계산
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Ratings for user i and item j
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # SGD to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
    
    # Test set 선정
    def set_test(self,ratings_test):
        test_set = []
        for i in range(len(ratings_test)):
            # ratings_test에서 index 뽑아오기 (0:유저, 1:아이템, 2:평점)
            x = self.user_id_index[ratings_test.iloc[i,0]]
            y = self.item_id_index[ratings_test.iloc[i,1]]
            z = ratings_test.iloc[i,2]
            test_set.append([x,y,z])
            # 해당 (사용자-아이템-평점)을 R에서 0으로 지운다 -> R을 사용해서 MF모델을 학습을 하기 때문에 test set은 R에서 제거해야 한다
            self.R[x,y] = 0
        self.test_set = test_set
        return test_set

    # Test set의 RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0],one_set[1])
            error += pow(one_set[2]-predicted,2)
        return np.sqrt(error/len(self.test_set))
    
    def test(self):
        self.P = np.random.normal(scale=1./self.k,size=(self.num_users,self.k))
        self.Q = np.random.normal(scale=1./self.k,size=(self.num_items,self.k))
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])
        
        row,columns = self.R.nonzero()
        self.samples = [(i,j,self.R[i,j]) for i,j in zip(row,columns)]
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            # train set의 rmse
            rmse1 = self.rmse()
            # test set의 rmse
            rmse2 = self.test_rmse()
            training_process.append((i+1,rmse1,rmse2))
            if self.verbose:
                if (i+1)%10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f"%(i+1,rmse1,rmse2))
        return training_process
    
    # 주어진 user_id와 item_id에 대한 예측치
    def get_one_prediction(self,user_id,item_id):
        return self.get_prediction(self.user_id_index[user_id],self.item_id_index[item_id])
    
    # 앞의 식 3번에 따라 모든 사용자의 모든 아이템에 대한 예측치(full matrix)를 계산해서 돌려준다
    def full_prediction(self):
        return self.b + self.b_u[:,np.newaxis] +self.b_d[np.newaxis,:] + self.P.dot(self.Q.T) # np.newaxis : 배열의 차원을 확장
    
# Testing MF RMSE
R_temp = ratings.pivot(index='user_id',columns='movie_id',values='rating').fillna(0)
mf = NEW_MF(R_temp,k=30,alpha=0.001,beta=0.02,iterations=100,verbose=True)
test_set = mf.set_test(ratings_test)
result = mf.test()

# 전체 예측치
print(mf.full_prediction())
# 개별 예측치
print(mf.get_prediction(1,2))

Iteration: 10 ; Train RMSE = 0.9659 ; Test RMSE = 0.9834
Iteration: 20 ; Train RMSE = 0.9410 ; Test RMSE = 0.9645
Iteration: 30 ; Train RMSE = 0.9297 ; Test RMSE = 0.9566
Iteration: 40 ; Train RMSE = 0.9230 ; Test RMSE = 0.9523
Iteration: 50 ; Train RMSE = 0.9182 ; Test RMSE = 0.9496
Iteration: 60 ; Train RMSE = 0.9142 ; Test RMSE = 0.9477
Iteration: 70 ; Train RMSE = 0.9104 ; Test RMSE = 0.9461
Iteration: 80 ; Train RMSE = 0.9062 ; Test RMSE = 0.9446
Iteration: 90 ; Train RMSE = 0.9009 ; Test RMSE = 0.9429
Iteration: 100 ; Train RMSE = 0.8940 ; Test RMSE = 0.9407
[[3.84431971 3.37073648 3.04207597 ... 3.35860338 3.48516608 3.48033164]
 [3.93026785 3.48943998 3.16275041 ... 3.43492616 3.55738317 3.5461305 ]
 [3.33560723 2.87669657 2.54092058 ... 2.82482555 2.93184838 2.91903706]
 ...
 [4.20574225 3.7631364  3.4497938  ... 3.71327137 3.82942411 3.81800725]
 [4.35370345 3.88428012 3.55513619 ... 3.83432788 3.95028469 3.94286003]
 [3.80612498 3.37875122 3.04470301 ... 3.28945754 3.4178910

### 연습문제

4.1 train/test set을 분리하는 방법을 shuffle() 대신에 앞장에서 사용한 train_test_split()을 사용해서 분리하도록 수정하고 실행해 보세요.